# <center>Part 1: Creating the Dask Cluster</center>

# Library imports

In [1]:
import dask
import dask.array as da
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
from climate.utils import load_dataset
from climate.kalman_filter import EnsembleKalmanFilter

/storage/homefs/ct19x463/envs/myenv/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


# Define Cluster
We here define the specifications for the individual workers in the cluster. In addition, we specify the queue for the jobs and the time allocated for each job (here two hours).

Note that this only defines the blueprint of the cluster, no actual resources are allocated here.

In [25]:
cluster = SLURMCluster(
    cores=4,                                                                
    memory="32 GB",                                                         
    death_timeout=6000,                                                                              
    walltime="02:00:00",                                                    
    job_extra=['--mem-per-cpu=12GB', '--qos="job_epyc2"', '--partition="epyc2"']
)

# Allocate Cluster 

We now create a client that will use the cluster. The *client* is the entity who will be in charge of submitting our computations to the cluster.

Finally, we allocate the resource by calling `client.scale(10)` which will allocate a cluster with 10 workers.

In [26]:
client = Client(cluster)
cluster.scale(10)

**Cluster Monitoring**: Note that the cluster automatically sets up an interactive monitoring dashboard available at port tcp/8787 of the scheduler. 

One can then set up a tunnel to that port of the scheduler to access the dashboard:

`ssh -N -f -L 8787:bnode034:8787 campusid@submit.unibe.ch`

Where `campusid` should be set to one's campus id login and `bnode034` should be set to the hostname of the scheduler (can be seen in the bash console of the scheduler).

In [16]:
cluster # We can also print cluster information from within the Jupyter console.
# Make sure to wait for the SLURM job scheduler to have time to allocate your workload.

SLURMCluster(14895422, 'tcp://10.1.3.15:15876', workers=12, threads=12, memory=89.40 GiB)

# <center>Part 2: Loading the Data</center>

Data loading is performed by the `load_zarr_dataset` function. The user only has to specify the location of the folder containing the data. 

Note that due to the amount of data for the different ensemble members, reading the whole dataset can be slow on systems with slow disks (as is painfully the case in UBELIX). 
To that end, the use can choose to only load the first `n` ensemble members to run quick tests. The user can also set `ìgnore_members` to true, 
which will return a fake dataset that has the same size as the full one (basically, for each ensemble member we use the ensemble mean). 
This functionality is useful when one wants to work with objects that have the same size as the full dataset but which can be loaded faster.

In [5]:
base_folder = "/storage/homefs/ct19x463/Dev/Climate/Data/"
TOT_ENSEMBLES_NUMBER = 3
    
# The loading function returns 4 datasets: the ensemble members, the ensemble
# mean, the instrumental data and the reference dataset.
(dataset_mean, dataset_members,
        dataset_instrumental, dataset_reference,
        dataset_members_zarr)= load_dataset(
        base_folder, TOT_ENSEMBLES_NUMBER, ignore_members=False)
print("Loading done.")

/storage/homefs/ct19x463/Dev/Climate/Data/Ensembles/Members/member_1/
/storage/homefs/ct19x463/Dev/Climate/Data/Ensembles/Members/member_2/
/storage/homefs/ct19x463/Dev/Climate/Data/Ensembles/Members/member_3/


/storage/homefs/ct19x463/envs/myenv/lib/python3.8/site-packages/xarray/coding/times.py:351: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
/storage/homefs/ct19x463/Dev/Climate/climate/utils.py:111: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  dataset_mean['time'] = dataset_mean.indexes['time'].to_datetimeindex()
/storage/homefs/ct19x463/Dev/Climate/climate/utils.py:112: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  dataset_members['time'] = dataset_members.indexes['tim

Loading done.


Note that the data loading does not put any actual data into memory. Also, the datasets returned by this method are quite raw to work with.

There are wrappers to ease data access. The easiest way to wrap data into an accessible format is to let a KalmanFilter object handle that for us under the hood.

In [27]:
from climate.kalman_filter import EnsembleKalmanFilter                                                                                                                                                         
my_filter = EnsembleKalmanFilter(dataset_mean, dataset_members_zarr, dataset_instrumental, client)

Maximal distance to matched point: 120.54565778878536 km.


**Remark:** Note that in the background the Kalman filter load

One can then easily get data vectors for a given time period using the `get_window_vector` function of the wrapped datasets. 
This function returns data vectors with the latitude, longitude and time dimensions concatenated in some fixed order.

In [ ]:
time_begin = '1961-01-16'
time_end = '1961-06-16'
my_filter.dataset_members.get_window_vector(time_begin, time_end)

<xarray.DataArray 'difference' (member_nr: 30, stacked_dim: 110592)>
dask.array<getitem, shape=(30, 110592), dtype=float32, chunksize=(1, 10000), chunktype=numpy.ndarray>
Coordinates:
    latitude   (stacked_dim) float64 dask.array<chunksize=(79488,), meta=np.ndarray>
    longitude  (stacked_dim) float64 dask.array<chunksize=(79488,), meta=np.ndarray>
  * member_nr  (member_nr) int64 1 2 3 4 5 6 7 8 9 ... 23 24 25 26 27 28 29 30
    time       (stacked_dim) datetime64[ns] dask.array<chunksize=(79488,), meta=np.ndarray>
Dimensions without coordinates: stacked_dim

In [ ]:
window_vector = client.scatter(my_filter.dataset_members.get_window_vector(time_begin, time_end))

# Getting the covariance is also simple.

In [28]:
cov = my_filter.get_ensemble_covariance(time_begin, time_end)
cov

/storage/homefs/ct19x463/envs/myenv/lib/python3.8/site-packages/dask/array/routines.py:328: PerformanceWarning: Increasing number of chunks by factor of 12
  intermediate = blockwise(


dask.array<truediv, shape=(110592, 110592), dtype=float32, chunksize=(10000, 10000), chunktype=numpy.ndarray>

One can trigger the computation of the covariance using `client.persist()`. This will keep the result distributed on the cluster. 
Note that full covariance computation takes less than 1 minute.

In [29]:
cov = client.persist(cov)

2022-03-30 11:54:33,613 - distributed.scheduler - ERROR - Error transitioning 'original-open_dataset-aeb882a396ff182f04673b7bc9d9ca23air_temperature-f7ea83e46dde6e2ffc915e453ffe60fa' from 'erred' to 'memory'
Traceback (most recent call last):
  File "/storage/homefs/ct19x463/envs/myenv/lib/python3.8/site-packages/distributed/scheduler.py", line 2321, in _transition
    assert not args and not kwargs, (args, kwargs, start_finish)
AssertionError: ((), {'worker': 'tcp://10.1.3.4:3242', 'nbytes': 48, 'typename': 'xarray.core.indexing.ImplicitToExplicitIndexingAdapter'}, ('erred', 'memory'))
2022-03-30 11:54:33,627 - distributed.utils - ERROR - ((), {'worker': 'tcp://10.1.3.4:3242', 'nbytes': 48, 'typename': 'xarray.core.indexing.ImplicitToExplicitIndexingAdapter'}, ('erred', 'memory'))
Traceback (most recent call last):
  File "/storage/homefs/ct19x463/envs/myenv/lib/python3.8/site-packages/distributed/utils.py", line 693, in log_errors
    yield
  File "/storage/homefs/ct19x463/envs/myenv

In [ ]:
cov[10, 10].compute()

In [ ]:
cluster

In [ ]:
client.shutdown